In [1]:
%reload_ext autoreload
%autoreload 2
import sys
sys.path.insert(1, '/home/a/a270230/LKF/lkf_detection/functions/')
from statistics_functions import *
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
res = '4km'

path_stat = '/work/bk1377/a270230/statistics/'
[years, lkfs, paths, paths_all] = np.load(path_stat + f'lkfs_paths_{res}.npy', allow_pickle=True)
LKFs = np.load(path_stat + f'LKFs_{res}.npy', allow_pickle=True)[0]

In [3]:
# this is only the growth rate between two time steps of the tracked lkfs.
# the growth rate of the individual lkfs is calculated first and then
# summed up. it is not actually a good metric for the total lkfs growth rate
for year in years:
    
    y = np.where(years==year)[0][0]
    for d in range(364):
        
        for i, lkf_path in enumerate(paths_all[y][d]):
            
            if np.size(lkf_path)>1:
                # if the lkf is tracked, set its growth rate to the difference
                # in lkf length between current and next time step
                LKFs[f'y{year}'][f'd{d}'].loc[i, 'growth rate'] =        \
                      LKFs[f'y{year}'][f'd{d+1}'].loc[lkf_path[1], 'length'] \
                    - LKFs[f'y{year}'][f'd{d}'].loc[i, 'length']
        
        # if the growth rate was not calculated (for untracked lkfs), set it to nan
        if 'growth rate' not in LKFs[f'y{year}'][f'd{d}'].keys():
            LKFs[f'y{year}'][f'd{d}']['growth rate'] = np.nan

    # also set the last day to nan
    LKFs[f'y{year}']['d364']['growth rate'] = np.nan
    
    # calculate the total growth rate by summing the growth rates of every tracked lkf
    total_growth_rate_y = []
    for day in range(365):
        total_growth_rate_y += np.nansum(LKFs[f'y{year}'][f'd{day}']['growth rate']),
    LKFs[f'y{year}']['total']['total growth rate'] = total_growth_rate_y

In [4]:
# collect all total growth rates in a single array, then calculate the interannual
# mean and standart deviation for the two time periods and write them into the dictionary
total_growth_rate = []
for year in years:
    total_growth_rate += LKFs[f'y{year}']['total']['total growth rate'],

for ystart, yend in zip([2013, 2093], [2020, 2100]):
    LKFs[f'{ystart} - {yend}']['total growth rate av'], LKFs[f'{ystart} - {yend}']['total growth rate sd'] = (
        av_sd(total_growth_rate, ystart, yend, years)
    )

In [5]:
# calculated in plot/ice_area_thickness.ipynb
if res == '4km': res_km = 4.337849218906646
if res == '1km': res_km = 1.083648783567869

# calculate the curvature of the lkfs according to
# curvature = 1 - distance_between_start_and_end_of_lkf / length_of_lkf
# a curvature of 0 thus corresponds to a straigth line
for year in years:
    y = np.where(years==year)[0][0]

    for d in range(365):

        curv = []
        if (len(lkfs[y][d])==0):
            # if no lkfs are detected for the current day, set it to nan
            LKFs[f'y{year}'][f'd{d}']['curvature'] = np.nan
        else:
            # calculate the curvature for the individual lkfs
            for i, lkf in enumerate(lkfs[y][d]):
                start = np.array([lkf[0,0], lkf[0,1]], dtype='int')
                end = np.array([lkf[-1,0], lkf[-1,1]], dtype='int')

                distance = np.sqrt( (end-start)[0]**2 + (end-start)[1]**2 ) * res_km

                LKFs[f'y{year}'][f'd{d}'].loc[i,'curvature'] = 1 - distance / LKFs[f'y{year}'][f'd{d}'].loc[i, 'length']

In [6]:
# calculate the mean curvature for each day
for year in years:
    for d in range(365):
        LKFs[f'y{year}']['total'].loc[d,'mean curvature'] = np.nanmean(LKFs[f'y{year}'][f'd{d}']['curvature'])

# collect all mean curvatures in a single array, then calculate the interannual
# mean and standart deviation for the two time periods and write them into the dictionary
mean_curvature = []
for year in years:
    mean_curvature += LKFs[f'y{year}']['total']['mean curvature'],

for ystart, yend in zip([2013, 2093], [2020, 2100]):
    LKFs[f'{ystart} - {yend}']['mean curvature av'], LKFs[f'{ystart} - {yend}']['mean curvature sd'] = (
        av_sd(mean_curvature, ystart, yend, years)
    )

In [7]:
np.save(path_stat + f'LKFs_{res}', [LKFs])